In [1]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_aed1318b5ff543f0af59c4425b8b6ab5 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='EwdAsr1iUBeawpjjpwM7UHV1R_UjD1ttmtDivWcJAEnF',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

streaming_body_2 = client_aed1318b5ff543f0af59c4425b8b6ab5.get_object(Bucket='ecgimagebasedheartbeatclassificat-donotdelete-pr-wxdfqhjgcngg8h', Key='Project.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [2]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_2.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

# Image Preprocessing

Importing Image Data Generator Library

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Configure Image datagenerator class

In [4]:
#setting parameter for image data agumentationto the training data
train_datagen=ImageDataGenerator(rescale=1./225,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
#image data agumenatation to test dataset
test_datagen=ImageDataGenerator(rescale=1./225)

Applying ImageDataGenerator Functionality to train and test Dataset

In [5]:
pwd

'/home/wsuser/work'

In [6]:
#performing data agumentation to train the dataset
x_train=train_datagen.flow_from_directory(directory=r'/home/wsuser/work/Project/data/train',target_size=(64,64),batch_size=32,class_mode='categorical')
#performing agumentation to test the dataset
x_test=test_datagen.flow_from_directory(directory=r'/home/wsuser/work/Project/data/test',target_size=(64,64),batch_size=32,class_mode='categorical')


Found 15341 images belonging to 6 classes.
Found 6825 images belonging to 6 classes.


# Model Building

Importing Libraries

In [7]:

import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

from tensorflow.keras.layers import Dense,Flatten

from tensorflow.keras.layers import Conv2D,MaxPooling2D
import tensorflow.keras

In [8]:
model= Sequential()

Adding CNN Layers

In [9]:
#adding model layer
model.add(Conv2D(32,(3,3),input_shape=(64,64,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())

Adding Dense Layers

In [10]:
model.add(Dense(32))
model.add(Dense(6,activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                200736    
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 1

Configuring the learning process

In [11]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

Train the Model

In [12]:
model.fit_generator(generator=x_train,steps_per_epoch = len(x_train), epochs=10, validation_data=x_test,validation_steps = len(x_test))

Epoch 1/10


/opt/conda/envs/Python-3.8-main/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1839: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


480/480 [==============================] - 71s 146ms/step - loss: 1.4969 - accuracy: 0.4647 - val_loss: 1.4700 - val_accuracy: 0.4779
Epoch 2/10
480/480 [==============================] - 68s 142ms/step - loss: 0.7270 - accuracy: 0.7619 - val_loss: 0.4592 - val_accuracy: 0.8705
Epoch 3/10
480/480 [==============================] - 68s 141ms/step - loss: 0.2766 - accuracy: 0.9200 - val_loss: 0.4883 - val_accuracy: 0.8588
Epoch 4/10
480/480 [==============================] - 68s 142ms/step - loss: 0.1973 - accuracy: 0.9424 - val_loss: 0.3067 - val_accuracy: 0.8977
Epoch 5/10
480/480 [==============================] - 70s 146ms/step - loss: 0.1595 - accuracy: 0.9537 - val_loss: 0.3031 - val_accuracy: 0.9074
Epoch 6/10
480/480 [==============================] - 69s 144ms/step - loss: 0.1404 - accuracy: 0.9581 - val_loss: 0.3816 - val_accuracy: 0.8923
Epoch 7/10
480/480 [==============================] - 66s 137ms/step - loss: 0.1219 - accuracy: 0.9625 - val_loss: 0.4366 - val_accuracy: 0.9

Saving the model

In [13]:
model.save('ECG.h5')

In [14]:
!tar -zcvf ECG-arrhythmia-classification-model_new.tgz ECG.h5

ECG.h5


In [15]:
ls -1

ECG-arrhythmia-classification-model_new.tgz
ECG.h5
__MACOSX/
my_model.tar.gz
Project/


In [16]:
!pip install watson-machine-learning-client --upgrade

In [17]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"N27QYBN9iREs6PUN9tg5bBuNObf3mQnmZ4AS5_WRgypC"
}
client=APIClient(wml_credentials)

In [18]:
def guid_from_space_name(client,space_name):
    space=client.spaces.get_details()
    #print(space)
    return(next(item for item in space ['resources']if item['entity']["name"]==space_name)['metadata']['id'])

In [19]:
space_uid=guid_from_space_name(client,'ECGClassification')
print("Space UID= "+space_uid)

Space UID= 84f6d295-4f9e-4ed1-aa4a-6a01eac78f9b


In [20]:
client.set.default_space(space_uid)

'SUCCESS'

In [21]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
scikit-learn_0.22-py3.6        154010fa-5b3b-4ac1-82af-4d5ee5abbc85  base
default_r3.6                   1b70aec3-ab34-4b87-8aa0-a4a3c8296a36  base
pytorch-onnx_1.3-py3.6         1bc6029

In [22]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_2.4-py3.7-horovod")
software_spec_uid

'1092590a-307d-563d-9b62-4eb7d64b3f22'

In [23]:
model_details = client.repository.store_model(model='ECG-arrhythmia-classification-model_new.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:"keras_2.2.4",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid})
model_id=client.repository.get_model_uid(model_details)

Note: Warnings!! :  Model type keras_2.2.4 is deprecated. We recommend you use a supported model type. See Supported Frameworks https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/pm_service_supported_frameworks.html


In [24]:
model_id

'474037e5-036c-466c-b33f-fcee1a6d4902'

In [25]:
#client.repository.download(model_id,'my_model.tar.gz')

Test the Model

In [26]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
model=load_model('/home/wsuser/work/Project/ECG.h5')
img=image.load_img("/home/wsuser/work/Project/uploads/PAC.png",target_size=(64,64))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
pred = model.predictECG(x)
y_pred=np.argmax(pred)
y_pred


2

In [27]:
index=['left Bundle Branch block','Normal','Premature Atrial Contraction','Premature Ventricular Contraction','Right Bundle Branch Block','Ventricular Fibrillation']
result = str(index[y_pred])
result

'Premature Atrial Contraction'